In [ ]:
%pip install mlflow
%pip install pymongo
%pip install xgboost

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
  Using cached xgboost-1.7.3-py3-none-manylinux2014_x86_64.whl (193.6 MB)
Python interpreter will be restarted.


In [ ]:
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
import mlflow.spark
from mlflow.models.signature import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
import pandas as pd 
from copy import copy
import cloudpickle

from pymongo import __version__ as mongo_version
from pymongo import MongoClient

import xgboost as xgb
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score, classification_report,roc_auc_score
import numpy as np                   # array, vector, matrix calculations
import pandas as pd                  # DataFrame handling

import os
import time

# method= SMOTE()

def get_mongo_conn():
    MONGO_CONN = "<connection_string>"
    database="fraud-detection"
    collection="usr_auth_rule_data"
    client = MongoClient(MONGO_CONN)
    db = client[database]
    collection = db[collection]
    return collection


In [ ]:
class RuleNMLModelWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.feature_columns = ["amt", "cat_id", "city_pop", "day", "hour", "lat", "long", "merch_lat", "merch_long", "month", "zip"]
        self.thresh = 0.3
        self.rules = [self.r1,self.r2,self.apply_model_pred]
        self.xgb_model = mlflow.pyfunc.load_model(f"models:/fraud_xgb_model_1/staging")

    def r1(self,x):
        if x['case']==0:
            return int(x['amt']>x['ACCT_AVL_CASH_BEFORE_AMT'])
        return x['case']
    
    def r2(self,x):
        if x['case']==0:
            return int(x['amt']>x['ACCT_CL_AMT'])
        return x['case']
    
    def apply_model_pred(self, x):
        if int(x['case'] == 0) | int(x['amt']>x['AVG_DLY_AUTHZN_AMT']):
            x = pd.DataFrame([x])
            x['city_pop'] = x['city_pop'].astype(np.int32)
            x['day'] = x['day'].astype(np.int32)
            x['hour'] = x['hour'].astype(np.int32)
            x['month'] = x['month'].astype(np.int32)
            x['zip'] = x['zip'].astype(np.int32)
            return int((self.xgb_model.predict(x)>self.thresh)[0])
        else:
            return x['case']

    def get_mongo_conn(self):
        MONGO_CONN = "<connection_string>"
        database="fraud-detection"
        collection="usr_auth_rule_data"
        client = MongoClient(MONGO_CONN)
        db = client[database]
        collection = db[collection]
        return collection       
        
    def predict(self, context, model_input):
        ip = copy(model_input)
        rules_df = pd.DataFrame(self.get_mongo_conn().find({},{"_id":0}))
        rules_df['cc_num'] = rules_df['cc_num'].apply(int)
        ip['cc_num'] = ip['cc_num'].apply(int)
        ip = ip.merge(rules_df,on='cc_num',how='left')
        ip['case'] = [0]*len(ip)
        for i in range(len(self.rules[:-1])):
            ip['case'] = ip.apply(self.rules[i],axis = 1)
            
        return np.asarray(list(ip['case'] | (self.xgb_model.predict(ip[self.feature_columns])>self.thresh).astype(int)))
        
        
#         return ip['case']

#         op = None
#         for col in [f"case{str(i)}" for i in range(len(self.rules))]:
#             if op is not None:
#                 op = op & ip[col]
#             else:
#                 op = ip[col]
#         ip['prediction'] = list(op)
#         ip['prediction'] = ip['prediction'].astype(int)
#         return (self.xgb_model.predict(ip[self.feature_columns])>self.thresh).astype(int)

In [ ]:
from databricks.feature_store import feature_table
from databricks.feature_store import FeatureStoreClient
from databricks.feature_store import FeatureLookup

fdf = spark.read.table("fraud_demo_txn_data").select("_id","is_fraud")
fdf.drop("is_fraud")

feature_table_name = "default.bfsi_txn_features"

feature_names = list(fdf.columns)

feature_lookups = [
    FeatureLookup(
        table_name=feature_table_name,
        lookup_key="_id",
    ),
]
fs = FeatureStoreClient()
training_set = fs.create_training_set(
    fdf,
    feature_lookups=feature_lookups,
    exclude_columns=["_id"],
    label="is_fraud",
)
training_df = training_set.load_df().toPandas()

In [ ]:
X = training_df.drop("is_fraud", axis=1)
y = training_df["is_fraud"]
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [ ]:
X.columns

Out[6]: Index(['age', 'amt', 'cat_id', 'cc_num', 'city_pop', 'day', 'gender_id',
       'hour', 'lat', 'long', 'merch_lat', 'merch_long', 'month', 'zip'],
      dtype='object')

In [ ]:
X_train

,age,amt,cat_id,cc_num,city_pop,day,gender_id,hour,lat,long,merch_lat,merch_long,month,zip
330201,24,14.29,11,6011693194885790,2607,28,0,20,45.6675,-93.2433,45.804080,-92.648545,1,55080
798518,28,67.93,7,3566875419572867,1654,4,1,3,44.6400,-73.3113,44.434532,-72.934131,10,5486
1260375,60,238.16,3,2297447006766555,478,18,0,13,39.9972,-88.6962,40.994808,-88.197438,5,61830
412511,43,11.40,6,2254917871818484,313,24,0,19,38.0828,-76.3477,37.275643,-77.219377,6,20687
344644,49,253.83,13,30029052116970,3833,2,1,20,42.1239,-79.1895,42.031912,-79.228261,3,14733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,47,53.18,11,630451534402,765,18,0,14,46.3535,-86.6345,45.506448,-86.192528,7,49895
259178,40,75.02,11,4451952084362894,686,13,1,22,34.5444,-102.1069,35.539703,-102.665203,3,79063
131932,29,117.01,8,4874006077381178,533,18,0,13,41.5646,-91.0859,41.779984,-91.361628,8,52760
671155,47,316.20,6,30270432095985,725,17,0,13,40.6761,-91.0391,40.278284,-90.167645,3,61454


In [ ]:
wrappedModel = RuleNMLModelWrapper()
wrappedModel.predict(None, X_train)

2023/02/14 12:03:00 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - scikit-learn (current: 1.2.1, required: scikit-learn==0.24.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
Out[9]: [0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 

In [ ]:
conda_env = mlflow.pyfunc.get_default_conda_env()
conda_env['dependencies'][2]['pip'] += ['xgboost']
conda_env['dependencies'][2]['pip'] += ['sklearn']
conda_env['dependencies'][2]['pip'] += ['imblearn']
conda_env['dependencies'][2]['pip'] += ['pymongo']

with mlflow.start_run(run_name='fraud_rule_model'):    
    collection = get_mongo_conn()
    wrappedModel = RuleNMLModelWrapper()
    signature = infer_signature(X_train, wrappedModel.predict(None, X_train))
    mlflow.pyfunc.log_model("fraud_rule_model", python_model=wrappedModel, conda_env=conda_env,signature=signature)

2023/02/14 12:21:45 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - scikit-learn (current: 1.2.1, required: scikit-learn==0.24.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.9/site-packages/mlflow/models/signature.py:130: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have mi

In [ ]:
run_id = mlflow.search_runs(filter_string='tags.mlflow.runName = "fraud_rule_model"').iloc[0].run_id
run_id

Out[9]: 'db03ae21909a4f3d933439b73e377631'

In [ ]:
model_name = "fraud_rule_model"
model_version = mlflow.register_model(f"runs:/{run_id}/fraud_rule_model", model_name)
 
# Registering the model takes a few seconds, so add a small delay
time.sleep(15)

Registered model 'fraud_rule_model' already exists. Creating a new version of this model...
2023/02/14 12:22:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: fraud_rule_model, version 9
Created version '9' of model 'fraud_rule_model'.


In [ ]:
from mlflow.tracking import MlflowClient
 
client = MlflowClient()
client.transition_model_version_stage(
  name=model_name,
  version=model_version.version,
  stage="Staging",
)

Out[11]: <ModelVersion: creation_timestamp=1676377354960, current_stage='Staging', description='', last_updated_timestamp=1676377376253, name='fraud_rule_model', run_id='db03ae21909a4f3d933439b73e377631', run_link='', source='dbfs:/databricks/mlflow-tracking/1071544389994397/db03ae21909a4f3d933439b73e377631/artifacts/fraud_rule_model', status='READY', status_message='', tags={}, user_id='1576958727467555', version='9'>

In [ ]:
1 and 0